In [ ]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.lines import Line2D
from math import radians, sin, cos, asin, sqrt
import re
import requests
from datetime import time, timedelta, datetime
from statistics import median, quantiles


import geohash2
import pyproj
from pyproj import Proj, transform, CRS
from functools import partial
import geopandas as gpd
from shapely.geometry import Point, Polygon
from geopy.distance import geodesic
from scipy.spatial import cKDTree
from scipy.stats import poisson

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from ortools.linear_solver import pywraplp

import random
from tqdm import tqdm
from time import sleep

# Data Loading

## Loading functions

In [ ]:
def calculate_demand(row):
    probabilities = {
        '0-10': 0,
        '10-20': 0.003458068783068975,
        '20-30': 0.013896825396825975,
        '30-40': 0.02136825396825512,
        '40-50': 0.02338333333333506,
        '50-60': 0.026277248677250214,
        '60-70': 0.03332089947090043,
        '70-80': 0.046515343915346036,
        '80+': 0.046515343915346036,
    }

    total_demand = 0
    for age_group, count in row['Alter'].items():
        lambda_value = count * probabilities[age_group]
        total_demand += poisson.rvs(lambda_value)
    return total_demand * 365


def setup_customer_data():
    customers_gdf = gpd.read_file('./Wuerzburg_Data/cluster_wuerzburg.gpkg')
    customers_gdf['Alter'] = customers_gdf['Alter'].apply(json.loads)

    customers_gdf['nachfrage'] = customers_gdf.apply(calculate_demand, axis=1)

    bevoelkerung_sum = customers_gdf[customers_gdf['cluster'] != -1].groupby('cluster')['sum_INSGESAMT_0'].sum()
    noise_demand = customers_gdf[customers_gdf['cluster'] == -1]['nachfrage'].iloc[0]

    # Verteilung des Noise-Bedarfs auf die anderen Cluster anteilig zur Anzahl der Polygone
    for cluster in bevoelkerung_sum.index:
        cluster_demand = noise_demand * (bevoelkerung_sum[cluster] / bevoelkerung_sum.sum())
        customers_gdf.loc[customers_gdf['cluster'] == cluster, 'nachfrage'] += np.round(cluster_demand)

    customers_gdf = customers_gdf[customers_gdf['cluster'] != -1].reset_index(drop=True)
    # Set Index to cluster id
    customers_gdf.set_index(['cluster'], inplace=True)

    return customers_gdf


def load_energy_costs():
    # Calculate timestamps (current time minus 48 hours and current time)
    current_time = datetime.now()
    past_timestamp = (current_time - timedelta(days=365)).timestamp() * 1000
    current_timestamp = current_time.timestamp() * 1000

    # Construct the API URL with updated timestamps
    api_url = f"https://api.awattar.de/v1/marketdata?start={past_timestamp}&end={current_timestamp}"

    # Send GET request using the requests library
    try:
        response = requests.get(api_url)
        response.raise_for_status()  # Raise an exception for non-2xx status codes
    except requests.exceptions.RequestException as error:
        print(f"Error: {error}")

    # Load data in json format
    data_energy = json.loads(response.content)
    # Extract market prices
    market_prices = [item["marketprice"] for item in data_energy["data"]]

    # Calculate quantiles
    q1, q2, q3 = quantiles(market_prices)  # Use quartiles function
    # Transform to Eur/kWh
    q1_kwh_eur = q1 / 1000
    q2_kwh_eur = q2 / 1000
    q3_kwh_eur = q3 / 1000

    print(f"Median market price (kWh): {q2_kwh_eur:.5f} Eur/kWh")
    print(f"25th percentile (Q1, kWh): {q1_kwh_eur:.2f} Eur/kWh")
    print(f"75th percentile (Q3, kWh): {q3_kwh_eur:.2f} Eur/kWh")

    return data_energy, q1_kwh_eur, q2_kwh_eur, q3_kwh_eur


def calculate_travel_distance(warehouses_gdf, customers_gdf):
    # Create an empty list to store the results
    data = []
    # Iterate through each warehouse in the warehouse DataFrame
    for warehouse_index, warehouse in warehouses_gdf.iterrows():
        for customer_index, customer in customers_gdf.iterrows():
            # Calculate the distance between the centroid of the region and the warehouse
            travel_distance = warehouse.geometry.distance(customer.geometry.centroid)/1000
            # Append the calculated values to the list
            data.append({'warehouse_id': warehouse_index, 'region_id': customer_index, 'travel_distance': travel_distance})
    # Create a DataFrame from the collected data
    df = pd.DataFrame(data)
    return df


def load_data(customers_gdf):
    
    # Grenzen des Simulationsrahmens laden
    geo_würzburg = gpd.read_file('./Wuerzburg_Data/geo_wuerzburg.gpkg')

    # Detaillierte Gebäude/Personen Daten laden
    bevölkerungs_gdf = gpd.read_file('./Wuerzburg_Data/pharmacy_assigned_complete.gpkg')
    bevölkerungs_gdf['Alter'] = bevölkerungs_gdf['Alter'].apply(json.loads)
    bevölkerungs_gdf['Geschlecht'] = bevölkerungs_gdf['Geschlecht'].apply(json.loads)

    # Apotheken Daten laden
    pharmacy_df = pd.read_csv('./Wuerzburg_Data/Würzburg-Apotheken.csv')
    pharmacy_gdf = gpd.GeoDataFrame(pharmacy_df, geometry=gpd.points_from_xy(pharmacy_df['lon'], pharmacy_df['lat']), crs=CRS("EPSG:4326"))
    pharmacy_gdf = pharmacy_gdf.to_crs(bevölkerungs_gdf.crs)

    # Warehouse Daten laden
    warehouses_gdf = gpd.read_file('./Wuerzburg_Data/warehouses_wuerzburg.gpkg')

    # Distanzmatrix der Cluster und Warehouses erstellen
    shifts_df = calculate_travel_distance(warehouses_gdf, customers_gdf)
    shifts_df.set_index(['warehouse_id', 'region_id'], inplace=True)

    #Loading the energy_data
    data_energy, q1_kwh_eur, q2_kwh_eur, q3_kwh_eur = load_energy_costs()

    return geo_würzburg, bevölkerungs_gdf, pharmacy_gdf, warehouses_gdf, shifts_df, data_energy, q1_kwh_eur, q2_kwh_eur, q3_kwh_eur


## Load the data

In [ ]:
customers_gdf = setup_customer_data()
geo_würzburg, bevölkerungs_gdf, pharmacy_gdf, warehouses_gdf, shifts_df, data_energy, q1_kwh_eur, q2_kwh_eur, q3_kwh_eur = load_data(customers_gdf)

# Plotting the Data

In [ ]:
# Rahmen der Grafik definieren
fig, ax = plt.subplots(figsize=(6, 6))
geo_würzburg.boundary.plot(ax=ax, color='gray', linewidth=0.5)

# Plotte gdf_loaded auf dieselbe Achse
pharmacy_gdf.plot(ax=ax, markersize=20)

# Titel hinzufügen
plt.title('Apothekenstandorte im Landkreis Würzburg')
plt.axis('off')

# Add scale bar
scalebar = ScaleBar(1, location='lower left', label='Maßstab')
ax.add_artist(scalebar)

# Zeige den Plot an
plt.show()

In [ ]:
# Create a colormap based on the "nachfrage" column
vmin = customers_gdf['nachfrage'].min()
vmax = customers_gdf['nachfrage'].max()
cmap = 'coolwarm'  

# Plot the raster with boundary
ax = geo_würzburg.boundary.plot(color='gray', linewidth=0.5, figsize = (5,5))

# Plot the customers with custom colormap
customers_gdf.plot(ax=ax, column='nachfrage', cmap=cmap, vmin=vmin, vmax=vmax)

# Add title and labels
plt.axis('off')
plt.title('Nachfrage der Cluster des Landkreis Würzburg')
#plt.xlabel('Breitengradkoordinate im CSR3035 Format')
#plt.ylabel('Längengradkoordinate im CSR3035 Format')

# Add colorbar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []  # Fake up the array of the scalar mappable
cbar = plt.colorbar(sm, ax=ax, shrink=0.5, label='Nachfrage')  # Adjust shrink value as needed

# Add scale bar
scalebar = ScaleBar(1, location='lower left', label='Maßstab')  # 1 pixel = 1 meter
ax.add_artist(scalebar)

plt.show()

In [ ]:
# Create a colormap based on the "total_price" column
vmin = warehouses_gdf['total_price_big'].min()
vmax = warehouses_gdf['total_price_big'].max()
cmap = 'coolwarm'  


# Rahmen der Grafik definieren
fig, ax = plt.subplots(figsize=(6, 6))
geo_würzburg.boundary.plot(ax=ax, color='gray', linewidth=0.5)

# Plotte gdf_loaded auf dieselbe Achse
warehouses_gdf.plot(ax=ax, column='total_price_big', cmap=cmap, markersize=20)

# Titel hinzufügen
plt.title('Lagerhaltungsimmobilien (Max) des Landkreis Würzburg')
plt.axis('off')

# Add colorbar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []  # Fake up the array of the scalar mappable
cbar = plt.colorbar(sm, ax=ax, shrink=0.5, label='Gesamter Mietpreis für ein Jahr')  # Adjust shrink value as needed

# Add scale bar
scalebar = ScaleBar(1, location='lower left', label='Maßstab')
ax.add_artist(scalebar)

# Zeige den Plot an
plt.show()

In [ ]:
# Create a colormap based on the "total_price" column
vmin = warehouses_gdf['total_price_small'].min()
vmax = warehouses_gdf['total_price_small'].max()
cmap = 'coolwarm'  


# Rahmen der Grafik definieren
fig, ax = plt.subplots(figsize=(6, 6))
geo_würzburg.boundary.plot(ax=ax, color='gray', linewidth=0.5)

# Plotte gdf_loaded auf dieselbe Achse
warehouses_gdf.plot(ax=ax, column='total_price_small', cmap=cmap, markersize=20)

# Titel hinzufügen
plt.title('Lagerhaltungsimmobilien (Min) des Landkreis Würzburg')
plt.axis('off')

# Add colorbar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []  # Fake up the array of the scalar mappable
cbar = plt.colorbar(sm, ax=ax, shrink=0.5, label='Gesamter Mietpreis für ein Jahr')  # Adjust shrink value as needed

# Add scale bar
scalebar = ScaleBar(1, location='lower left', label='Maßstab')
ax.add_artist(scalebar)

# Zeige den Plot an
plt.show()

In [ ]:
# Convert JSON data to pandas DataFrame
df_energy = pd.DataFrame(data_energy["data"])

# Convert timestamps to datetime format
df_energy["start_timestamp"] = pd.to_datetime(df_energy["start_timestamp"], unit="ms")
df_energy["end_timestamp"] = pd.to_datetime(df_energy["end_timestamp"], unit="ms")

# Filter data for values greater than 0
df_filtered = df_energy[df_energy["marketprice"] > 0]

# Plot market price vs timestamps (using filtered data)
plt.figure(figsize=(12, 6))  # Adjust figure size as needed
plt.plot(df_filtered["start_timestamp"], df_filtered["marketprice"], label="Market Price (Eur/MWh)")

# Set labels and title
plt.xlabel("Date")
plt.ylabel("Market Price (Eur/MWh)")
plt.title("Market Price Fluctuations (Values > 0)")  # Update title

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Set y-axis limits to start from 0 (optional)
plt.ylim(bottom=0)  # This ensures the y-axis starts at 0

# Add gridlines
plt.grid(True)

# Show legend
plt.legend()

plt.tight_layout()
plt.show()

# Optimierungsmodel

## Optimierung

In [ ]:
def optimize(W,
             R,
             S,
             warehouses_gdf_run,
             cost_per_km_drone,
             factory_setup_costs, 
             qm_per_customer, 
             minimum_square_requirement, 
             rent_factor, 
             max_flight_distance, 
             drone_initial_costs, 
             drone_speed, 
             time_window, 
             night_shift_dist, 
             demand_factor, 
             drone_puffer):
    
    M = 1000000000
    
    # Create a solver
    solver = pywraplp.Solver('FacilityLocation', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    # Define decision variables
    # Which warehouse serves which region
    x = {}
    for w, r in S:
        x[w,r] = solver.BoolVar(name=f'x_{w}_{r}')

    # Which warehouses are opened
    y = {}
    # How many drones are needed in each warehouse
    z = {}
    # How much space is rented in each warehouse
    d = {}
    for w in W:
        y[w] = solver.BoolVar(name=f'y_{w}')
        z[w] = solver.IntVar(0, solver.infinity(), name=f'z_{w}')
        d[w] = solver.IntVar(0, solver.infinity(), name=f'd_{w}')
        
    # Objective Function
    objective = solver.Objective()
    
    
    # Fixed costs for opening warehouses
    for w in W:
        objective.SetCoefficient(y[w], factory_setup_costs)  # Add fixed factory setup costs
        objective.SetCoefficient(d[w], warehouses_gdf_run.loc[w, 'pricePerSquareMetre'] * rent_factor)  # Cost per square meter

        
    # Costs for acquiring drones
    for w in W:
        objective.SetCoefficient(z[w], drone_initial_costs)
    
    # Variable costs for transportation
    for w, r in S:
        objective.SetCoefficient(x[w,r], cost_per_km_drone * shifts_df.loc[w,r].travel_distance * 2 * customers_gdf.loc[r, 'nachfrage'] * demand_factor)

    objective.SetMinimization()
    
    # Constraints
    # Regions can only be served by open warehouses
    for w in W:
        for r in R:
            solver.Add(x[w,r] <= y[w])

    # Each region has to be served by exactly one warehouse
    for r in R:
        solver.Add(solver.Sum(x[w,r] for w in W) == 1)

    # Each warehouse needs to be assigned with a certain amount of drones
    # for w in W:
    #     total_travel_time = solver.Sum(x[w, r] * shifts_df.loc[w, r].travel_distance * 2 * drone_puffer * (((customers_gdf.loc[r, 'nachfrage'] * (1 - night_shift_dist)) / 365) / drone_speed) for r in R) / time_window
    #     solver.Add(total_travel_time <= z[w])

    daily_demand_factor = (1 - night_shift_dist) / 365
    for w in W:
        # Initialize the total travel time expression
        total_time = solver.Sum(
            x[w, r] * shifts_df.loc[w, r].travel_distance * 2 * drone_puffer *
            (customers_gdf.loc[r, 'nachfrage'] * daily_demand_factor / drone_speed) 
            for r in R
        ) / time_window
        solver.Add(total_time <= z[w]) # Add the constraint

    # Each warehouse is assigned a certain amound of space that is between the boundries of the offering
    for w in W:
        solver.Add(y[w] * warehouses_gdf_run.loc[w, 'floorSpace_small'] <= d[w])
        solver.Add(y[w] * warehouses_gdf_run.loc[w, 'floorSpace_big'] >= d[w])
        
    # The distance from warehouse to customer can't be taller than the maximum flight range of each drone
    for w in W:
        for r in R:
            solver.Add(x[w,r] * shifts_df.loc[w,r].travel_distance <= max_flight_distance)

    # Each warehouse has to fulfill a certain minimum space requirement
    for w in W:
        solver.Add(y[w] * minimum_square_requirement <= warehouses_gdf_run.loc[w, 'floorSpace_small'])

    # Each warehouse needs a certain amount of space for each customer served
    for w in W:
        customer_demand_sum = solver.Sum(x[w, r] * customers_gdf.loc[r, 'sum_INSGESAMT_0'] for r in R)
        solver.Add((customer_demand_sum / 12 * qm_per_customer) <= y[w] * warehouses_gdf_run.loc[w, 'floorSpace_small'])

    return solver, x, y, z, d

## Lösungsausgabe

In [ ]:
def solve(W, R, solver, x, y, z, d, customers_gdf_run, warehouses_gdf_run):
  opened_warehouses = []
  customers_gdf_run['assigned_warehouse'] = 0
  warehouses_gdf_run['number_of_drones'] = 0
  warehouses_gdf_run['floor_space_assigned'] = 0

  #Solving the problem
  status = solver.Solve()
  print('Solved!')

  def print_solution(status, solver, opened_warehouses, x, y, z, d, W, R):

    if status == pywraplp.Solver.OPTIMAL:
      print("Objective value:", solver.Objective().Value())
      print("Opened warehouses:")
      opened_warehouses.clear()  # Clear the list before appending
      for w in W:
        if y[w].solution_value() > 0.5:
          opened_warehouses.append(w)
          warehouses_gdf_run.loc[w, 'number_of_drones'] = z[w].solution_value()
          warehouses_gdf_run.loc[w, 'floor_space_assigned'] = d[w].solution_value()
          print(f"- Warehouse {w}")
          print(f"Floor-Space: {d[w].solution_value()}")
          print(f"Drones needed: {z[w].solution_value()}")
      print("Warehouse assignments:")
      for r in R:
        assigned_warehouse = None
        for w in W:
          if x[w, r].solution_value() > 0.5:
            assigned_warehouse = w
            break
        if assigned_warehouse:
          customers_gdf_run.loc[r, 'assigned_warehouse'] = assigned_warehouse
          print(f"- Region {r} served by warehouse {assigned_warehouse}")
          distance = shifts_df.loc[assigned_warehouse, r].travel_distance
          print(f"- Distance: {distance}")
        else:
          print(f"- Region {r} has no assigned warehouse (might be infeasible)")
    else:
      print("Solver failed to find an optimal solution. Status:", status)


  print_solution(status, solver, opened_warehouses, x, y, z, d, W, R)
  
  return opened_warehouses, customers_gdf_run, warehouses_gdf_run

## Solution Graphics

In [ ]:
#Rahmen der Grafik definieren
ax = geo_würzburg.boundary.plot(color='gray', linewidth=0.5, figsize = (7,7))

# Plotte gdf_loaded auf dieselbe Achse
warehouses_gdf[~warehouses_gdf.index.isin(opened_warehouses)].plot(ax=ax, color='red', marker='o', markersize=25)
warehouses_gdf.loc[opened_warehouses].plot(ax=ax, color='blue', marker='^', markersize=35)

# Titel hinzufügen
plt.title('Lagerhaltungsimmobilien des Landkreis Würzburg')

plt.axis('off')

# Add scale bar
scalebar = ScaleBar(1, location='lower left', label='Maßstab')
ax.add_artist(scalebar)

# Add legend
custom_legend = [Line2D([0], [0], marker='^', color='w', markerfacecolor='blue', markersize=10, label='Geöffnet'),
                 Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, label='Nicht geöffnet')]
plt.legend(handles=custom_legend, title='Warenhäuser', bbox_to_anchor=(1,0.95), loc='upper left', borderaxespad=0., fontsize='small', ncol=1)
plt.tight_layout()

# # Index jedes Warehouses anzeigen
# for idx, row in warehouses_gdf.loc[opened_warehouses].iterrows():
#     plt.annotate(idx, (row.geometry.x, row.geometry.y), xytext=(5, 5), textcoords='offset points', fontsize=8, color='black')

# for idx, row in warehouses_gdf[~warehouses_gdf.index.isin(opened_warehouses)].iterrows():
#     plt.annotate(idx, (row.geometry.x, row.geometry.y), xytext=(5, 5), textcoords='offset points', fontsize=8, color='black')

# Zeige den Plot an
plt.show()

In [ ]:
# Create a colormap with a color for each opened warehouse
colors = plt.cm.tab20(np.linspace(0, 1, len(opened_warehouses)))
warehouse_colors = dict(zip(opened_warehouses, colors))

# Create a dictionary to map each warehouse to a color
warehouse_colors = {
    warehouse: color
    for warehouse, color in zip(opened_warehouses, warehouse_colors.values())
}

# Plot the boundaries of the region
ax = geo_würzburg.boundary.plot(color='gray', linewidth=0.5, figsize=(8, 8))


# Plot the customers with the color of their assigned warehouse
for index, row in customers_gdf.iterrows():
    warehouse = row['assigned_warehouse']
    color = warehouse_colors.get(warehouse, 'gray')  # Use gray color if warehouse is not in the dictionary
    customers_gdf.iloc[[index]].plot(ax=ax, color=color, markersize=20)


# Plot the warehouses with their assigned colors
for warehouse, color in warehouse_colors.items():
    warehouses_gdf[warehouses_gdf.index == warehouse].plot(ax=ax, color=color, marker='^', markersize=50, edgecolor='black')


# Turn off axis
plt.axis('off')

# Add title
plt.title('Lagerhaltungsimmobilien des Landkreis Würzburg')

# Add legend
legend_handles = [Line2D([0], [0], marker='^', color='w', markerfacecolor=color, markersize=10, label=warehouse) for warehouse, color in warehouse_colors.items()]
ax.legend(handles=legend_handles, title='Warenhäuser', bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0., fontsize='small', ncol=2)

# Add scale bar
scalebar = ScaleBar(1, location='lower left', label='Maßstab')
ax.add_artist(scalebar)

# Show the plot
plt.show()

## Assigning a warehouse to each building based on the optimal value

In [ ]:
# Funktion, um das zugewiesene Lager für einen Punkt zu finden
def find_assigned_warehouse(point, customers_gdf_run, cluster_sindex):
    # Räumlichen Index für das Cluster-GDF erstellen
    possible_matches_index = list(cluster_sindex.intersection(point.bounds))
    possible_matches = customers_gdf_run.iloc[possible_matches_index]
    output = possible_matches[possible_matches.geometry.contains(point)]
    if not output.empty:
        return [output.assigned_warehouse.iloc[0]]
    else:
        nearest_polygon_index = cluster_sindex.nearest(point)[0]
        nearest_polygon = customers_gdf_run.iloc[nearest_polygon_index]
        return [nearest_polygon.assigned_warehouse.iloc[0]]

In [ ]:
def assign_warehouses(bevölkerungs_gdf_run, customers_gdf_run):
    # Verfolgen Sie den Fortschritt der apply-Methode
    tqdm.pandas()
    
    cluster_sindex = customers_gdf_run.sindex

    # Die apply-Methode auf die GeoDataFrame anwenden, um das zugewiesene Lager für jeden Punkt zu finden
    warehouses = bevölkerungs_gdf_run['geometry'].progress_apply(find_assigned_warehouse, customers_gdf_run = customers_gdf_run, cluster_sindex = cluster_sindex)

    bevölkerungs_gdf_run['assigned_warehouse'] = 0
    bevölkerungs_gdf_run['distance_warehouse'] = 0

    for index, row in tqdm(bevölkerungs_gdf_run.iterrows(), total=len(bevölkerungs_gdf_run)):
        bevölkerungs_gdf_run.loc[index, 'assigned_warehouse'] = warehouses[index][0]
        warehouse_geometry = warehouses_gdf.loc[row['assigned_warehouse'], 'geometry']
        population_geometry = row['geometry']
        bevölkerungs_gdf_run.loc[index, 'distance_warehouse'] = warehouse_geometry.distance(population_geometry) / 1000

    return bevölkerungs_gdf_run

## Simulations Loop

In [ ]:
def calculate_demand(row, probabilities):
    total_demand = 0
    for age_group, count in row['Alter'].items():
        lambda_value = count * probabilities[age_group]
        total_demand += poisson.rvs(lambda_value)
    return total_demand

In [ ]:
def assign_random_timestamp(row, night_shift_dist, start_time, end_time):
    timestamp_list = []
    for i in range(row['nachfrage']): 
        probabilities_timestamp = [1 - night_shift_dist, night_shift_dist]  # Wahrscheinlichkeit für innerhalb und außerhalb der Öffnungszeiten

        start_hour = start_time.hour
        start_minute = start_time.minute
        end_hour = end_time.hour
        end_minute = end_time.minute

        if np.random.choice([False, True], p=probabilities_timestamp):
            # Außerhalb der Öffnungszeiten
            if random.choice([True, False]):
                # Vor den Öffnungszeiten
                hour = random.randint(0, start_hour - 1)
                minute = random.randint(0, 59)
                second = random.randint(0, 59)
            else:
                # Nach den Öffnungszeiten
                hour = random.randint(end_hour + 1, 23)
                minute = random.randint(0, 59)
                second = random.randint(0, 59)
        else:
            # Innerhalb der Öffnungszeiten
            if start_hour == end_hour:
                hour = start_hour
                minute = random.randint(start_minute, end_minute)
            else:
                hour = random.randint(start_hour, end_hour)
                if hour == start_hour:
                    minute = random.randint(start_minute, 59)
                elif hour == end_hour:
                    minute = random.randint(0, end_minute)
                else:
                    minute = random.randint(0, 59)
            second = random.randint(0, 59)
        
        timestamp = datetime.combine(datetime.today(), datetime.min.time()) + timedelta(hours=hour, minutes=minute, seconds=second)
        timestamp_list.append(timestamp)
    return timestamp_list

In [ ]:
def calculate_penalty_costs(bevoelkerungs_gdf_sim, warehouses_gdf_sim, drone_speed, time_window):
    # Convert time_window to a timedelta object
    time_window = timedelta(minutes=time_window)

    # Initialize a dictionary to keep track of next available times for drones in each warehouse
    warehouse_drones = {warehouse: [datetime.min] * warehouses_gdf_sim.loc[warehouse, 'number_of_drones'] for warehouse in warehouses_gdf_sim.index}

    total_exceeded_minutes = 0

    # Flatten the demand timestamps and associate them with their warehouses and distances
    all_demands = []
    for index, row in bevoelkerungs_gdf_sim.iterrows():
        assigned_warehouse = row['assigned_warehouse']
        distance_warehouse = row['distance_warehouse']
        for timestamp in row['demand_timestamp']:
            all_demands.append((assigned_warehouse, distance_warehouse, timestamp))

    # Sort all demands by timestamp
    all_demands.sort(key=lambda x: x[2])

    # Process each demand
    for assigned_warehouse, distance_warehouse, timestamp in all_demands:

        # Find the first available drone
        available_drones = warehouse_drones[assigned_warehouse]
        next_available_time = min(available_drones)
        start_time = max(timestamp, next_available_time)  # Use start_time to calculate when the drone can actually start

        # Calculate the delivery time
        delivery_time = start_time + timedelta(minutes=distance_warehouse / drone_speed) 

        # Check if the delivery time exceeds the time window
        if (delivery_time - timestamp) > time_window:
            exceeded_minutes = ((delivery_time - timestamp) - time_window).total_seconds() / 60
            total_exceeded_minutes += exceeded_minutes

        # Update the next available time for the drone
        return_trip_end_time = delivery_time + timedelta(minutes=distance_warehouse / drone_speed)
        available_drones[available_drones.index(next_available_time)] = return_trip_end_time

    return total_exceeded_minutes

In [ ]:
def build_trip_request_string(pharmacy_lon, pharmacy_lat, demands):
    waypoints = f"{pharmacy_lon},{pharmacy_lat}"
    for _, row in demands.iterrows():
        waypoints += f";{row.lon},{row.lat}"
    return f"http://router.project-osrm.org/trip/v1/driving/{waypoints}?roundtrip=true&source=first&destination=last&overview=false&steps=false"

def calculate_total_distance_and_duration(demands, pharmacy_lon, pharmacy_lat, total_distance_sum, total_duration_sum, counter):
    try:
        request_string = build_trip_request_string(pharmacy_lon, pharmacy_lat, demands)
        res = requests.get(request_string)
        res.raise_for_status()  # Raise an exception for non-200 status codes

        content = json.loads(res.content)

        # Check if trips are available
        if 'trips' in content and len(content['trips']) > 0:
            trip = content['trips'][0]
            total_distance = trip['distance']
            total_duration = trip['duration']
            return total_distance, total_duration
        else:
            #print(f"No trips found for request {request_string}")
            return total_distance_sum/counter, total_duration_sum/counter

    except requests.exceptions.RequestException as e:
        #print(f"Error occurred on attempt for request {request_string}: {e}")
        return total_distance_sum/counter, total_duration_sum/counter

def expand_timestamps(demands_df):
    # Explode the DataFrame by demand_timestamp
    demands_df = demands_df.explode('demand_timestamp')
    demands_df['demand_timestamp'] = pd.to_datetime(demands_df['demand_timestamp'])
    return demands_df

def calculate_pharmacy_routing(demands_df, pharmacies_df):
    # Expand the demands DataFrame so each timestamp is in its own row
    demands_df = expand_timestamps(demands_df)

    # Split demands into two DataFrames by half, grouped by assigned pharmacy
    demands_df_groups = demands_df.groupby('assigned_pharmacy')
    groups_list = list(demands_df_groups)
    split_index = len(groups_list) // 2

    # Split the DataFrame into two DataFrames
    first_half_demands = pd.concat([group[1] for group in groups_list[:split_index]])
    second_half_demands = pd.concat([group[1] for group in groups_list[split_index:]])
    

    # Group by assigned pharmacy and filter demands by time
    before_13pm = first_half_demands[first_half_demands['demand_timestamp'].dt.hour < 13].drop_duplicates(subset=['lon', 'lat'])
    after_13pm = first_half_demands[first_half_demands['demand_timestamp'].dt.hour >= 13].drop_duplicates(subset=['lon', 'lat'])

    total_distance_sum = 0
    total_duration_sum = 0
    counter = 0

    # Vectorized processing within each time period
    for time_period, demands in [('before_13pm', before_13pm), ('after_13pm', after_13pm)]:
        if demands.empty:
            continue

        grouped = demands.groupby('assigned_pharmacy')

        for pharmacy_id, group in grouped:
            pharmacy_info = pharmacies_df[pharmacies_df['id'] == pharmacy_id]

            if pharmacy_info.empty:
                print(f"Pharmacy ID {pharmacy_id} not found in pharmacies dataframe.")
                continue

            pharmacy_lon = pharmacy_info.lon.iloc[0]
            pharmacy_lat = pharmacy_info.lat.iloc[0]

            total_distance, total_duration = calculate_total_distance_and_duration(group, pharmacy_lon, pharmacy_lat, total_distance_sum, total_duration_sum, counter)
            total_distance_sum += total_distance
            total_duration_sum += total_duration
            counter += 1
            

    
    second_half_demands = second_half_demands.groupby('assigned_pharmacy')
    for pharmacy_id, group in second_half_demands:
        pharmacy_info = pharmacies_df[pharmacies_df['id'] == pharmacy_id]

        if pharmacy_info.empty:
            print(f"Pharmacy ID {pharmacy_id} not found in pharmacies dataframe.")
            continue

        pharmacy_lon = pharmacy_info.lon.iloc[0]
        pharmacy_lat = pharmacy_info.lat.iloc[0]

        total_distance, total_duration = calculate_total_distance_and_duration(group, pharmacy_lon, pharmacy_lat, total_distance_sum, total_duration_sum, counter)
        total_distance_sum += total_distance
        total_duration_sum += total_duration
        counter += 1

    return total_distance_sum / 1000, total_duration_sum / 60

In [ ]:
def simulate(opened_warehouses_run, bevölkerungs_gdf_run, warehouses_gdf_run, cost_per_km_drone, drone_speed, time_window, start_time, end_time, night_shift_dist, cost_per_km_car, factory_setup_costs, drone_initial_costs):

    # Wahrscheinlichkeitsverteilung
    probabilities_demand = {
        '0-10': 0,
        '10-20': 0.003458068783068975,
        '20-30': 0.013896825396825975,
        '30-40': 0.02136825396825512,
        '40-50': 0.02338333333333506,
        '50-60': 0.026277248677250214,
        '60-70': 0.03332089947090043,
        '70-80': 0.046515343915346036,
        '80+': 0.046515343915346036,
    }

    #Calculate the monthly fix costs due to drone and factory setup
    rental_cost = np.sum(warehouses_gdf_run.loc[opened_warehouses_run].floor_space_assigned * warehouses_gdf_run.loc[opened_warehouses_run].pricePerSquareMetre) / 12
    factory_setup_cost = (len(opened_warehouses_run) * factory_setup_costs) / 12
    drone_setup_cost = (np.sum(warehouses_gdf_run.loc[opened_warehouses_run].number_of_drones) * drone_initial_costs) / 12 

    drone_transportation_cost = 0
    drone_transportation_time = 0
    time_penalty_costs = 0

    car_transportation_cost_customer = 0
    car_transportation_time_customer = 0

    car_transportation_cost_pharmacy = 0
    car_transportation_time_pharmacy = 0


    tqdm.pandas()

    #Loop der Simulation über ein gesamtes Jahr
    for i in range(1):

        print(f'Simulation - Tag: {i + 1}')
        
        
        bevölkerungs_gdf_run['nachfrage'] = 0
        bevölkerungs_gdf_run['nachfrage'] = bevölkerungs_gdf_run.progress_apply(calculate_demand, axis=1, probabilities = probabilities_demand)
        bevölkerungs_gdf_run['demand_timestamp'] = bevölkerungs_gdf_run.progress_apply(lambda row: assign_random_timestamp(row, night_shift_dist, start_time, end_time), axis=1)

        # Calculate transportation costs and time using vectorized operations
        drone_transportation_cost += np.sum(bevölkerungs_gdf_run['nachfrage'] * bevölkerungs_gdf_run['distance_warehouse'] * cost_per_km_drone * 2)
        drone_transportation_time += np.sum((bevölkerungs_gdf_run['nachfrage'] * bevölkerungs_gdf_run['distance_warehouse']) / drone_speed )
        time_penalty_costs += calculate_penalty_costs(bevölkerungs_gdf_run, warehouses_gdf_run, drone_speed, time_window)

        car_transportation_cost_customer += np.sum(bevölkerungs_gdf_run['nachfrage'] * bevölkerungs_gdf_run['distance_pharmacy'] * cost_per_km_car * 2)
        car_transportation_time_customer += np.sum(bevölkerungs_gdf_run['nachfrage'] * bevölkerungs_gdf_run['time_pharmacy'])

        car_transportation_cost_pharmacy_temp, car_transportation_time_pharmacy_temp = calculate_pharmacy_routing(bevölkerungs_gdf_run[bevölkerungs_gdf_run['nachfrage'] > 0], pharmacy_gdf)
        car_transportation_cost_pharmacy += car_transportation_cost_pharmacy_temp
        car_transportation_time_pharmacy += car_transportation_time_pharmacy_temp

    return [rental_cost, factory_setup_cost, drone_setup_cost, drone_transportation_cost, drone_transportation_time, time_penalty_costs, car_transportation_cost_customer, car_transportation_time_customer, car_transportation_cost_pharmacy, car_transportation_time_pharmacy]

In [ ]:
def print_cost_summary(costs):

  print("-" * 50)
  print("Logistical Cost Summary (per year):")
  print("-" * 50)
  print(f"Factory Rental Cost: \t€{costs[0]:.2f}")
  print(f"Factory Setup Cost: \t€{costs[1]:.2f}")
  print(f"Drone Setup Cost: \t\t\t€{costs[2]:.2f}")
  print("-" * 50)
  print(f"Drone Transportation Cost: \t€{costs[3]:.2f}")
  print(f"Drone Transportation Time: \t\t\t€{costs[4]/60:.2f} hours")
  print(f"Transportation Penalty Cost: \t\t\t€{costs[5]:.2f}")
  print("-" * 50)
  print(f"Car/Truck Transportation Cost: \t€{costs[6]:.2f}")
  print(f"Car/Truck Transportation Time: \t{costs[7]/60:.2f} hours")
  print("-" * 50)

# Robustheit / Parameter Changes

## Run the sim

In [ ]:
def calculate_times(minutes):
  start_time = time(hour=8, minute=0) # Set initial start time to 8:00 AM
  total_minutes = (start_time.hour * 60 + start_time.minute) + minutes # Convert minutes to total number of minutes
  end_time = time(hour=total_minutes // 60 % 24, minute=total_minutes % 60)   # Calculate end time by handling overflow within 24 hours

  # Adjust start time if end time is before start time (overflow)
  if end_time < start_time:
    # Calculate the adjustment needed (difference in minutes)
    if start_time.minute > end_time.minute:
      start_time = time(start_time.hour - end_time.hour, start_time.minute - end_time.minute)
    else:
      start_time = time(start_time.hour - end_time.hour -1, 60 - end_time.minute)
    end_time = time(23,59,59)

  return start_time, end_time

In [ ]:
def run(parameter_values, bevölkerungs_gdf_run, warehouses_gdf_run, customers_gdf_run, shifts_df_run):

    battery_capacity_wh = parameter_values['battery_capacity_mAh'] * parameter_values['battery_voltage']
    battery_capacity_j = battery_capacity_wh * 3600  # J # Convert battery capacity from Wh to J
    Epm = battery_capacity_j / (parameter_values['max_flight_distance'] * 1000)  # J/m # Energy per meter in Joules
    kWh_km = Epm * 1000 / 3600000  # kWh/km # Convert energy per meter to kWh per kilometer
    cost_per_km_drone = parameter_values['kwh_eur'] * kWh_km  # EUR/km # Calculate cost per kilometer for the drone
    
    W = warehouses_gdf_run.index.values
    R = customers_gdf_run.index.values
    S = shifts_df_run.index.values

    result = []
    
    # Set the optimization problem
    solver, x, y, z, d = optimize(
        W = W, 
        R = R,
        S = S, 
        warehouses_gdf_run = warehouses_gdf_run,
        cost_per_km_drone = cost_per_km_drone,
        factory_setup_costs = parameter_values['factory_setup_costs'],
        qm_per_customer = parameter_values['qm_per_customer'],
        minimum_square_requirement = parameter_values['minimum_square_requirement'],
        rent_factor = parameter_values['rent_factor'],
        max_flight_distance = parameter_values['max_flight_distance'],
        drone_initial_costs = parameter_values['drone_initial_costs'],
        drone_speed = parameter_values['drone_speed'],
        time_window = parameter_values['time_window'],
        night_shift_dist = parameter_values['night_shift_dist'],
        demand_factor = parameter_values['demand_factor'],
        drone_puffer = parameter_values['drone_puffer'])

    print('Solver set up!')

    # Solve the problem and get the solution
    opened_warehouses, customers_gdf_run, warehouses_gdf_run = solve(
        W = W,
        R = R, 
        solver = solver, 
        x = x, 
        y = y, 
        z = z, 
        d = d, 
        customers_gdf_run = customers_gdf_run, 
        warehouses_gdf_run = warehouses_gdf_run)
    
    # Assign the in the solution chosen warehouses in the dataset
    bevölkerungs_gdf_run = assign_warehouses(bevölkerungs_gdf_run, customers_gdf_run)
    print('Dataset set up!')

    # Get the start and end time of the service hours
    start_time, end_time = calculate_times(minutes=parameter_values['time_window'])

    # Car/Truck Parameters:
    cost_per_km_car = 0.38

    # Simulate with optimal values
    costs = simulate(
        opened_warehouses_run = opened_warehouses, 
        bevölkerungs_gdf_run = bevölkerungs_gdf_run, 
        warehouses_gdf_run = warehouses_gdf_run, 
        cost_per_km_drone = cost_per_km_drone,
        start_time = start_time,
        end_time = end_time,
        drone_speed = parameter_values['drone_speed'],
        time_window = parameter_values['time_window'],
        night_shift_dist = parameter_values['night_shift_dist'],
        cost_per_km_car = cost_per_km_car,
        factory_setup_costs = parameter_values['factory_setup_costs'],
        drone_initial_costs = parameter_values['drone_initial_costs'])
    print('Simulation done!')

    print_cost_summary(costs= costs)

    result.append({
    'opened_warehouses': opened_warehouses,
    'number_of_drones': np.sum(warehouses_gdf_run.loc[opened_warehouses].number_of_drones),
    'floor_space_assigned': np.sum(warehouses_gdf_run.loc[opened_warehouses].floor_space_assigned),
    'objective_value': solver.Objective().Value(),
    'costs': costs
    })

    return result        

## Sensitivity Analysis

In [ ]:
def sensitivity_analysis(parameter_values, number_of_runs, sensitivity_index, parameter_names):
    
    if sensitivity_index != 'None':
        triangular_values = np.random.triangular(parameter_values[sensitivity_index][0], parameter_values[sensitivity_index][1], parameter_values[sensitivity_index][2], number_of_runs)
    
    bevölkerungs_gdf_run = bevölkerungs_gdf
    warehouses_gdf_run = warehouses_gdf
    customers_gdf_run = customers_gdf
    shifts_df_run = shifts_df

    result_list = []

    for i in range(number_of_runs):
    
        if sensitivity_index != 'None':
            parameter_values[sensitivity_analysis][1] = triangular_values[i]

        base_values = [param[1] for param in parameter_values]
        
        base_dict = {
            'factory_setup_costs': base_values[0],
            'qm_per_customer': base_values[1],
            'minimum_square_requirement': base_values[2],
            'rent_factor': base_values[3],
            'max_flight_distance': base_values[4],
            'drone_initial_costs': base_values[5],
            'drone_speed': base_values[6],
            'time_window': base_values[7],
            'night_shift_dist': base_values[8],
            'demand_factor': base_values[9],
            'drone_puffer': base_values[10],
            'battery_capacity_mAh': base_values[11],
            'battery_voltage': base_values[12],
            'kwh_eur': base_values[13]
            }

        results = run(base_dict, bevölkerungs_gdf_run, warehouses_gdf_run, customers_gdf_run, shifts_df_run)

        if sensitivity_index != 'None':
            result_list.append({
                'parameter': parameter_names[sensitivity_index],
                'parameter_value': triangular_values[i],
                'results': results
            })
        else: 
            result_list.append({
                'parameter': 'Base Value',
                'parameter_value': 'Base_Value',
                'results': results
            })

    
    return result_list

# Complete Loop

In [ ]:
def find_min_travel_distance(shifts_df):
    # Initialize variables
    min_distance = 0
    max_distance = shifts_df['travel_distance'].max()
    shifts_df.reset_index(inplace=True)

    while min_distance <= max_distance:

        current_distance = min_distance + 0.1

        # Assuming 'node_a' and 'node_b' are columns containing node IDs, and 'distance' is the distance column
        filtered_data = shifts_df[shifts_df['travel_distance'] < current_distance]

        all_regions = shifts_df.region_id.unique()
        filtered_regions = filtered_data.region_id.unique()  
        region_present = set(all_regions).issubset(set(filtered_regions))

        if region_present:
            break
        else:
            min_distance = current_distance  # Update min distance if condition not met

    shifts_df.set_index(['warehouse_id', 'region_id'], inplace=True)
    return min_distance  # Minimum distance found

In [ ]:
parameter_names = [
    "factory_setup_costs",
    "qm_per_customer",
    "minimum_square_requirement",
    "rent_factor",
    "max_flight_distance",
    "drone_initial_costs",
    "drone_speed",
    "time_window",
    "night_shift_dist",
    "demand_factor",
    "drone_puffer",
    "battery_capacity_mAh",
    "battery_voltage",
    "kwh_eur"
]

In [ ]:
base_factory_setup_costs = 100000 #Eur
min_factory_setup_costs = 100000 # Set an appropriate value
max_factory_setup_costs = 100000 # Set an appropriate value

base_qm_per_customer = 0.05 #Quadratmeter
min_qm_per_customer = 0.05 # Set an appropriate value
max_qm_per_customer = 0.05 # Set an appropriate value

base_minimum_square_requirement = 50 #Quadratmeter
min_minimum_square_requirement = 50 # Set an appropriate value
max_minimum_square_requirement = 50 # Set an appropriate value

base_rent_factor = 1 #Prozent
min_rent_factor = 1 # Set an appropriate value
max_rent_factor = 1 # Set an appropriate value

base_max_flight_distance = 20 #km
min_max_flight_distance = 20 # Set an appropriate value
max_max_flight_distance = 20 # Set an appropriate value

base_drone_initial_costs = 4000 #Eur
min_drone_initial_costs = 4000 # Set an appropriate value
max_drone_initial_costs = 4000 # Set an appropriate value

base_drone_speed = 80/60 #km/h/60 = km/min
min_drone_speed = 80/60 # Set an appropriate value
max_drone_speed = 80/60 # Set an appropriate value

base_time_window = 630 #Min
min_time_window = 630 # Set an appropriate value
max_time_window = 630 # Set an appropriate value

base_night_shift_dist = 0.00100142348 #Prozent
min_night_shift_dist = 0.00100142348 # Set an appropriate value
max_night_shift_dist = 0.00100142348 # Set an appropriate value

base_demand_factor = 1 #Prozent
min_demand_factor = 1 # Set an appropriate value
max_demand_factor = 1 # Set an appropriate value

base_drone_puffer = 2
min_drone_puffer = 2 # Set an appropriate value
max_drone_puffer = 2 # Set an appropriate value

base_battery_capacity_mAh = 4000
min_battery_capacity_mAh = 4000 # Set an appropriate value
max_battery_capacity_mAh = 4000 # Set an appropriate value

base_battery_voltage = 11.1
min_battery_voltage = 11.1 # Set an appropriate value
max_battery_voltage = 11.1 # Set an appropriate value

base_kwh_eur = q2_kwh_eur
min_kwh_eur = q2_kwh_eur # Set an appropriate value
max_kwh_eur = q2_kwh_eur # Set an appropriate value

In [ ]:
parameter_values = [
    [min_factory_setup_costs, base_factory_setup_costs, max_factory_setup_costs],
    [min_qm_per_customer, base_qm_per_customer, max_qm_per_customer],
    [min_minimum_square_requirement, base_minimum_square_requirement, max_minimum_square_requirement],
    [min_rent_factor, base_rent_factor, max_rent_factor],
    [min_max_flight_distance, base_max_flight_distance, max_max_flight_distance],
    [min_drone_initial_costs, base_drone_initial_costs, max_drone_initial_costs],
    [min_drone_speed, base_drone_speed, max_drone_speed],
    [min_time_window, base_time_window, max_time_window],
    [min_night_shift_dist, base_night_shift_dist, max_night_shift_dist],
    [min_demand_factor, base_demand_factor, max_demand_factor],
    [min_drone_puffer, base_drone_puffer, max_drone_puffer],
    [min_battery_capacity_mAh, base_battery_capacity_mAh, max_battery_capacity_mAh],
    [min_battery_voltage, base_battery_voltage, max_battery_voltage],
    [min_kwh_eur, base_kwh_eur, max_kwh_eur]
]

In [ ]:
complete_analysis = []

for i in range(len(parameter_values)):
    complete_analysis.append(sensitivity_analysis(parameter_values, 1, 'None', parameter_names))
    complete_analysis.append(sensitivity_analysis(parameter_values, 1, i, parameter_names))

# Tests

In [ ]:
max_flight_distance = 20 #km
qm_per_customer = 0.05 #Quadratmeter
minimum_square_requirement = 50 #Quadratmeter
factory_setup_costs = 100000 #Eur
drone_initial_costs = 4000 #Eur
drone_speed = 80/60 #km/h/60 = km/min
time_window = 630 #Min
night_shift_dist = 0.00100142348 #Prozent
demand_factor = 1 #Prozent
rent_factor = 1 #Prozent
drone_puffer = 2

battery_capacity_mAh = 40000
voltage = 11.1
battery_capacity_wh = battery_capacity_mAh * voltage
battery_capacity_j = battery_capacity_wh * 3600  # J # Convert battery capacity from Wh to J
Epm = battery_capacity_j / (max_flight_distance * 1000)  # J/m # Energy per meter in Joules
kWh_km = Epm * 1000 / 3600000  # kWh/km # Convert energy per meter to kWh per kilometer
cost_per_km_drone = q2_kwh_eur * kWh_km  # EUR/km # Calculate cost per kilometer for the drone


# Car/Truck Parameters:
cost_per_km_car = 0.38



W = warehouses_gdf.index.values
R = customers_gdf.index.values
S = shifts_df.index.values

# Set the optimization problem
solver, x, y, z, d = optimize(W,
                              R,
                              S,
                              warehouses_gdf,
                              cost_per_km_drone,
                              factory_setup_costs,
                              qm_per_customer, 
                              minimum_square_requirement, 
                              rent_factor, 
                              max_flight_distance, 
                              drone_initial_costs, 
                              drone_speed, 
                              time_window, 
                              night_shift_dist, 
                              demand_factor, 
                              drone_puffer
                              )
print('Solver set up!')

# Solve the problem and get the solution
opened_warehouses, customers_gdf, warehouses_gdf  = solve(W, R, solver, x, y, z, d, customers_gdf, warehouses_gdf)


In [ ]:
bevölkerungs_gdf = assign_warehouses(bevölkerungs_gdf, customers_gdf)

In [ ]:
time_window = 630
start_time = time(8, 0)
end_time = time(18, 30)

In [ ]:
bevölkerungs_gdf, costs = simulate(opened_warehouses_run=opened_warehouses,
                 bevölkerungs_gdf_run=bevölkerungs_gdf,
                 warehouses_gdf_run=warehouses_gdf,
                 cost_per_km_drone=cost_per_km_drone,
                 drone_speed=drone_speed,
                 time_window=time_window,
                 start_time = start_time,
                 end_time= end_time,
                 night_shift_dist=night_shift_dist,
                 cost_per_km_car=cost_per_km_car,
                 factory_setup_costs=factory_setup_costs,
                 drone_initial_costs=drone_initial_costs)

In [ ]:
car_transportation_cost_customer = 0
car_transportation_time_customer = 0
drone_transportation_cost = 0
drone_transportation_time = 0

In [ ]:
car_transportation_cost_customer = np.sum(bevölkerungs_gdf['nachfrage'] * bevölkerungs_gdf['distance_pharmacy'] * cost_per_km_car * 2)
car_transportation_time_customer = np.sum(bevölkerungs_gdf['nachfrage'] * bevölkerungs_gdf['time_pharmacy'] / 60)
drone_transportation_cost = np.sum(bevölkerungs_gdf['nachfrage'] * bevölkerungs_gdf['distance_warehouse'] * cost_per_km_drone * 2)
drone_transportation_time = np.sum((bevölkerungs_gdf['nachfrage'] * bevölkerungs_gdf['distance_warehouse']) / drone_speed )
print(car_transportation_cost_customer, car_transportation_time_customer)
print(drone_transportation_cost, drone_transportation_time)

## Get the data straight

In [ ]:
test = gpd.read_file('./Donner_Data/pharmacy_assigned_complete.gpkg')

In [ ]:
test['time_pharmacy']